<a href="https://colab.research.google.com/github/yeyevtushenko/AI/blob/Lesson11.03.2024/Lesson11_03_2024_AI_P_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Used%20Car%20Dataset.csv", index_col="Unnamed: 0")
df.drop(columns=['car_name', 'registration_year', 'ownsership', 'transmission'], inplace=True)

Дані про ціни вживаних автомобілів у Індії
* car_name - назва авто у форматі 'рік компанія основна назва'
* registration_year - місяць та рік реєстрації
* ownsership - кількість власників у форматі 'Second owner'
* transmission - тип коробки передач
* insurance_validity - тип страховки
* fuel_type - вид палива
* seats - кількість сидінь
* kms_driven - пробіг
* manufacturing_year - рік виробництва, має тип object, потрібно застосувати astype(int)
* mileage(kmpl) - скільки кілометрів проїде за літр палива
* engine(cc) - об'єм двигуна у мл
* max_power(bhp) - потужність у кінських силах
* torque(Nm) - крутний момент двигуна
* price(in lakhs) - ціна у сто тисяч рупій

[Повний аналіз даних](https://www.kaggle.com/code/abdelrasoul/used-cars-prices-prediction)

In [4]:
df.head()

,insurance_validity,fuel_type,seats,kms_driven,manufacturing_year,mileage(kmpl),engine(cc),max_power(bhp),torque(Nm),price(in lakhs)
0,Comprehensive,Petrol,5,56000,2017,7.81,2996.0,2996.0,333.0,63.75
1,Comprehensive,Petrol,5,30615,2020,17.40,999.0,999.0,9863.0,8.99
2,Comprehensive,Diesel,5,24000,2018,20.68,1995.0,1995.0,188.0,23.75
3,Comprehensive,Petrol,5,18378,2019,16.50,1353.0,1353.0,13808.0,13.56
4,Comprehensive,Petrol,5,44900,2019,14.67,1798.0,1798.0,17746.0,24.00


In [5]:
df = df[['fuel_type', 'kms_driven', 'engine(cc)', 'price(in lakhs)']]
df.head()

,fuel_type,kms_driven,engine(cc),price(in lakhs)
0,Petrol,56000,2996.0,63.75
1,Petrol,30615,999.0,8.99
2,Diesel,24000,1995.0,23.75
3,Petrol,18378,1353.0,13.56
4,Petrol,44900,1798.0,24.00


# Завдання 1
Очистіть дані від викидів, розділіть на тренувальну та тестову чатини

In [6]:
df.shape

(1553, 4)

In [32]:
def remove_outliers(df, column_name):
  column = df[column_name]

  q1 = column.quantile(q=0.25)
  q3 = column.quantile(q=0.75)

  iqr = q3 - q1

  lower = q1 - 1.5*iqr
  upper = q3 + 1.5*iqr

  mask = (column >= lower) & (column <= upper)

  return df[mask]

for column_name in df.select_dtypes(include="number").columns:
  df = remove_outliers(df, column_name)

In [8]:
df.shape

(1223, 4)

In [9]:
y = df['price(in lakhs)']
X = df.drop(columns='price(in lakhs)')

In [10]:
X = pd.get_dummies(X)
X.head()

,kms_driven,engine(cc),fuel_type_5 Seats,fuel_type_CNG,fuel_type_Diesel,fuel_type_Petrol
1,30615,999.0,0,0,0,1
2,24000,1995.0,0,0,1,0
3,18378,1353.0,0,0,0,1
4,44900,1798.0,0,0,0,1
5,42000,1199.0,0,0,0,1


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [12]:
df.dtypes

fuel_type           object
kms_driven           int64
engine(cc)         float64
price(in lakhs)    float64
dtype: object

# Завдання 2
Створіть Pipeline для обробки числових стовпчиків(заповнення пропусків)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [14]:
num_columns = X.select_dtypes(include="number").columns

num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")
    )])

# Завдання 3
Створіть Pipeline для обробки категоріальних стовпчиків(заповнення пропусків та кодування)

In [15]:
cat_columns = X.select_dtypes(include="object").columns

cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OneHotEncoder(handle_unknown='ignore'))])

# Завдання 4
Об'єднайте попередні Pipeline в один ColumnTransformer

In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ])

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['kms_driven', 'engine(cc)', 'fuel_type_5 Seats', 'fuel_type_CNG',
       'fuel_type_Diesel', 'fuel_type_Petrol'],
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index([], dtype='object'))])

# Завдання 5
Створіть остаточну модель та натренуйте її

In [17]:
from sklearn.linear_model import LinearRegression

model = Pipeline(
    [("prep", preprocessor),
     ("model", LinearRegression())
     ]
)

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)', 'fuel_type_5 Seats', 'fuel_type_CNG',
       'fuel_type_Diesel', 'fuel_type_Petrol'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index([], dtype='object'))])),
                ('model', LinearRegression())])

In [18]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)', 'fuel_type_5 Seats', 'fuel_type_CNG',
       'fuel_type_Diesel', 'fuel_type_Petrol'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index([], dtype='object'))])),
                ('model', LinearRegression())])

In [34]:
y_pred = model.predict(X_test)

# Завдання 6
Виведіть основні метрики моделі

In [35]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["MSE", name] = metrics.mean_squared_error(y_true, y_pred)
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [36]:
y_pred = model.predict(X_test)

In [37]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

In [38]:
all_metrics

,train,test
MAE,3.86,3.67
MSE,30.00,29.05
R2,0.33,0.34


# Завдання 7
Збережіть модель

In [39]:
data = pd.DataFrame({'fuel_type': ['Diesel'],
                     'kms_driven': [48000],
                     'engine(cc)': [1832]})

data

,fuel_type,kms_driven,engine(cc)
0,Diesel,48000,1832


In [40]:
import joblib

joblib.dump(model, 'new_model.joblib')

new_model = joblib.load('new_model.joblib')

new_model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)', 'fuel_type_5 Seats', 'fuel_type_CNG',
       'fuel_type_Diesel', 'fuel_type_Petrol'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index([], dtype='object'))])),
                ('model', LinearRegression())])

In [41]:
new_model.predict(data)

KeyError: "['fuel_type_5 Seats', 'fuel_type_CNG', 'fuel_type_Diesel', 'fuel_type_Petrol'] not in index"